# <center>Site-level Study on EU ETS Policy Effects:</center>
## <center>Dual-Outcome Analysis (ETS CO₂ + Satellite NOx)</center>

## Research question / Hypotheses

**Core question:**
> How do EU ETS carbon market stringency interventions affect both verified CO₂ emissions and satellite-derived NOx emission proxies around major industrial emitters?

**Dual Outcomes:**
1. **ETS CO₂** (ground-truth): Verified annual emissions from EU ETS registry
2. **Satellite NOx** (proxy): Beirle-style flux-divergence NOx estimates from TROPOMI

**Sub-questions / hypotheses:**
- *H1:* Allocation shortfall (ratio < 1) induces facilities to reduce combustion, lowering both CO₂ and NOx.
- *H2:* The magnitude of emission reduction correlates with plant characteristics (fuel type, capacity) and local geography.
- *H3:* Treatment effects may differ between ground-truth CO₂ and noisy satellite NOx due to measurement error attenuation.

## Variables and Causal Structure

We work with a **plant–year panel** indexed by facility *i* and year *t*.

### Dual Outcomes
- **$Y^{CO2}_{it}$**: Verified annual CO₂ emissions from EU ETS registry (tCO₂/yr, log-transformed).
  Ground-truth administrative data with high reliability.
- **$Y^{NOx}_{it}$**: Satellite-derived NOx emission proxy (kg/s) via Beirle-style flux-divergence.
  Noisy but independent measure of combustion intensity.

### Treatment (Causal Target)
- **Policy $P_{it}$**: EU ETS stringency at the plant–year level, measured by the
  allocation ratio (free allocation / verified emissions). Values < 1 indicate the
  facility must purchase additional allowances, creating abatement incentives.

### Observed Covariates
- **Plant characteristics $X_{it}$**: Time-varying capacity (MW) and fuel mix shares.
  Used as controls for **BOTH outcomes**.
- **Geographic context $G_i$**: High-dimensional AlphaEarth embeddings (64-dim) encoding
  land use, infrastructure, and climate patterns. Used as controls for **NOx only** (see below).

### Unobserved Variables
- **$U_i$**: Plant-level time-invariant unobservables (baseline technology, combustion efficiency).
- **$U_{it}$**: **Plant-level time-varying unobservables**—the key identification challenge.
  Includes dispatch/utilization, maintenance status, operational efficiency changes.
- **$U_{rt}$**: Region–time effects (electricity demand, fuel prices, regional policy enforcement).

### Why Dual Outcomes?

1. **ETS CO₂** is ground-truth but may be subject to reporting incentives
2. **Satellite NOx** is independent but noisy (~35-45% uncertainty)
3. Consistent effects across both outcomes strengthen causal claims
4. Divergent effects may reveal measurement error attenuation in NOx

### Why AlphaEarth Embeddings Are Used Only for NOx

| Outcome | Use Embeddings? | Rationale |
|---------|-----------------|-----------|
| **ETS CO₂** | ❌ No | Administrative mass balance data. Geographic context does NOT affect measurement. Confounders absorbed by Facility FE (time-invariant) and Region×Year FE (time-varying). |
| **Satellite NOx** | ✅ Yes | Satellite retrieval quality varies with terrain (AMF corrections), land use (urban background), and climate (atmospheric conditions). Embeddings control for this measurement heterogeneity. |

This aligns with the DAG: $G_i \to Y^{NOx}_{it}$ only, not $G_i \to Y^{CO2}_{it}$.

```mermaid
flowchart LR
  %% === UNOBSERVED ===
  subgraph Unobserved
    direction TB
    Ui["U_i: time-invariant (technology, efficiency)"]
    Uit["U_it: time-varying (dispatch, maintenance)"]
    Urt["U_rt: region–time (demand, prices)"]
  end

  %% === OBSERVED ===
  subgraph Observed Controls
    direction TB
    X["X_it: capacity, fuel mix"]
    G["G_i: AlphaEarth embeddings"]
    W["W_it: wind (measurement only)"]
  end

  %% === TREATMENT ===
  subgraph Treatment
    A["A_it: Free allocation (predetermined Feb)"]
    P["P_it: Allocation ratio = A/Y"]
  end

  %% === DUAL OUTCOMES ===
  subgraph "Dual Outcomes"
    Y_CO2["Y^CO2_it: Verified ETS emissions"]
    Y_NOx["Y^NOx_it: Satellite NOx proxy"]
  end

  %% === CAUSAL ARROWS ===

  %% Time-invariant unobserved → absorbed by facility FE
  Ui --> X
  Ui --> Y_CO2
  Ui --> Y_NOx
  Ui -.->|"absorbed by facility FE"| A

  %% Plant-level time-varying unobserved (KEY CHALLENGE)
  Uit --> Y_CO2
  Uit --> Y_NOx
  Urt --> Uit

  %% Region-time effects → absorbed by Region×Year FE
  Urt -.->|"absorbed by region×year FE"| A
  Urt --> Y_CO2
  Urt --> Y_NOx

  %% Observed controls
  X --> Y_CO2
  X --> Y_NOx
  
  %% AlphaEarth: affects satellite retrieval quality (background, terrain AMF)
  %% Does NOT affect administrative ETS reporting
  G --> Y_NOx
  
  %% Wind: ONLY affects satellite measurement (advection calculation)
  %% Does NOT affect ETS verified emissions (administrative data)
  W --> Y_NOx

  %% === TREATMENT PATHWAY (NO CYCLE) ===
  %% Key insight: A_it is PREDETERMINED (granted by Feb 28)
  %% Emissions Y_it occur DURING the year, AFTER allocation is known
  %% So: A_it → P_it → Y_it (no reverse causation in same period)
  
  A --> P
  Y_CO2 -.->|"ex-post denominator"| P
  
  %% CAUSAL EFFECTS OF INTEREST
  P ==>|"β_CO2 (causal)"| Y_CO2
  P ==>|"β_NOx (causal)"| Y_NOx

  %% Mediation: policy affects dispatch (potential mechanism)
  P -.->|"mediation via dispatch"| Uit
```

---

## Breaking the Apparent Cycle: EU ETS Compliance Calendar

The DAG above shows no cycle between $P_{it}$ and $Y_{it}$. This relies on the **temporal structure** of EU ETS compliance:

| Event | Timing | Implication |
|-------|--------|-------------|
| Free allocation granted | **28 February** of year $t$ | $A_{it}$ is predetermined before operations |
| Emissions occur | During year $t$ | Facilities know $A_{it}$ when making dispatch decisions |
| Emissions verified & reported | 31 March of year $t+1$ | $Y_{it}$ measured ex-post |
| Allowances surrendered | **30 April** of year $t+1$ | Compliance deadline |

> *Source*: European Commission, "ETS Revision: no change to deadline to surrender allowances in 2023" (Jan 2023): "The deadline for competent authorities to grant free allocation is [28 February]... the deadline to surrender allowances [is 30 April]."

**Why no cycle**: The allocation $A_{it}$ for year $t$ is **known before** emissions decisions are made. The allocation ratio $R_{it} = A_{it}/Y_{it}$ captures policy stringency, where:
- **Numerator** ($A_{it}$): Predetermined based on historical benchmarks
- **Denominator** ($Y_{it}$): Endogenous, but this is the **response** we're measuring

The dashed arrow $Y^{CO2}_{it} \dashrightarrow P_{it}$ represents the mechanical ex-post calculation of the ratio, not reverse causation. The causal direction is unambiguous: facilities observe $A_{it}$ at the start of the year and adjust operations accordingly.

---

## Why AlphaEarth and Wind Affect Only NOx

| Variable | Affects ETS CO₂? | Affects Satellite NOx? | Reason |
|----------|------------------|------------------------|--------|
| **AlphaEarth** $G_i$ | ❌ No | ✅ Yes | ETS is administrative data; satellite retrieval depends on terrain, land use (AMF), urban background |
| **Wind** $W_{it}$ | ❌ No | ✅ Yes | ETS is reported mass balance; Beirle flux-divergence uses wind for advection calculation |
| **Capacity/Fuel** $X_{it}$ | ✅ Yes | ✅ Yes | Both outcomes reflect combustion intensity |
| **Dispatch** $U_{it}$ | ✅ Yes | ✅ Yes | Both outcomes respond to operational activity |

---

## Identification Strategy

### What We Control For

| Variable | Absorbed By | Rationale |
|----------|-------------|-----------|
| $U_i$ (time-invariant) | Facility FE | Technology, location, baseline efficiency |
| $U_{rt}$ (region-time) | Region×Year FE | Demand shocks, fuel prices, regional policy |
| $X_{it}$ (observed) | Controls | Capacity, fuel mix |
| $G_i$ (geography) | AlphaEarth embeddings | Satellite retrieval context (NOx only) |

### What We Do NOT Control For: $U_{it}$

Plant-level time-varying unobservables ($U_{it}$) include dispatch, maintenance, and efficiency changes.
We deliberately do not control for these because:

1. **Dispatch as confounder**: Regional demand → higher dispatch → more emissions → lower $R_{it}$.
   Same demand → more combustion → higher NO₂. This creates spurious correlation.

2. **Dispatch as mediator**: Policy affects dispatch via carbon costs in bids (higher costs → higher bids →
   lower dispatch probability). Thus $P_{it} \to U_{it} \to Y_{it}$. Controlling for $U_{it}$ blocks
   this pathway and biases $\hat{\beta}$ toward zero.

3. **Facility×Year FE infeasible**: Would absorb all within-facility-year variation, including treatment.

### Our Solution: Region×Year FE

Region×Year FE absorbs the *common regional component* of $U_{it}$ (since dispatch responds to
regional demand/prices) without estimating facility-specific parameters. The identifying variation:

> *Within the same region and year, do facilities with different allocation ratios show different emissions?*

This leaves **facility-specific deviations** in $U_{it}$ as residual confounding (e.g., idiosyncratic
outages, plant-specific demand). These are plausibly second-order and orthogonal to allocation ratio
conditional on capacity and fuel mix controls.

---

## Analysis Methodology

### Treatment Definition

**Continuous Treatment**: `eu_alloc_ratio = allocated_allowances / verified_emissions`
- Ratio < 1 → Facility must purchase additional allowances (treated)
- Ratio ≥ 1 → Facility has sufficient free allocation (control/less treated)

**Discrete Treatment** (for Callaway-Sant'Anna):
- `treated = 1` if `eu_alloc_ratio < 1` in a given year
- `cohort` = first year when `treated = 1` (staggered adoption)

### Clustering Strategy

We use **NUTS2 regions** (Eurostat administrative units) for spatial clustering:
1. **Clustered standard errors** — accounts for within-region correlation in errors
2. **Region × Year fixed effects** — absorbs region-specific time shocks (electricity demand, fuel prices, policy enforcement)

**Note**: PyPSA-Eur power system clusters are used for **electricity sector heterogeneity analysis only** (not for main specification).

### Six Core Specifications

| # | Outcome | Embedding Strategy | Estimator |
|---|---------|-------------------|-----------|
| **1** | ETS CO₂ | None | TWFE |
| **2** | ETS CO₂ | None | Callaway-Sant'Anna |
| **3** | Satellite NOx | PCA (10 dims) | TWFE |
| **4** | Satellite NOx | PCA (10 dims) | Callaway-Sant'Anna |
| **5** | Satellite NOx | PLS (10 dims) | TWFE |
| **6** | Satellite NOx | PLS (10 dims) | Callaway-Sant'Anna |

**Notes:**
- ETS CO₂ uses **no embeddings** (administrative data unaffected by geography)
- Satellite NOx uses AlphaEarth embeddings to control for retrieval heterogeneity
- **PCA**: Unsupervised projection (outcome-agnostic)
- **PLS**: Supervised projection (trained on facility-level mean NOx for causal validity)

### Heterogeneity Analysis

Treatment effects may vary systematically across:
- **Electricity sector**: `is_electricity` (EU ETS activity codes 1 or 20 = combustion)
- **Urbanization**: Urban vs rural facilities (background NO₂ differs)
- **Fuel type**: Coal vs gas vs other (abatement costs differ)
- **Capacity**: Large vs small facilities (economies of scale in abatement)
- **Country**: Cross-country policy enforcement differences

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# =============================================================================
# COLUMN CONSTANTS
# =============================================================================
FAC_ID_COL = "idx"
YEAR_COL = "year"
CLUSTER_COL = "nuts2_region"
PYPSA_CLUSTER_COL = "pypsa_cluster"
ALLOC_RATIO_COL = "eu_alloc_ratio"
ETS_CO2_COL = "eu_verified_tco2"
LOG_ETS_CO2_COL = "log_ets_co2"
NOX_OUTCOME_COL = "beirle_nox_kg_s"
NOX_SE_COL = "beirle_nox_kg_s_se"
IN_URBAN_AREA_COL = "in_urban_area"
URBANIZATION_DEGREE_COL = "urbanization_degree"
IS_ELECTRICITY_COL = "is_electricity"

# =============================================================================
# DATA PATHS (adjust if running from different directory)
# =============================================================================
DATA_DIR = Path("../data/out")
STATIC_PATH = DATA_DIR / "facilities_static.parquet"
YEARLY_PATH = DATA_DIR / "facilities_yearly.parquet"
BEIRLE_PATH = DATA_DIR / "beirle_panel.parquet"

# =============================================================================
# ANALYSIS PARAMETERS
# =============================================================================

# Time range
START_YEAR = 2018
END_YEAR = 2023

# Treatment
TREATMENT_COL = ALLOC_RATIO_COL        # "eu_alloc_ratio"
TREATMENT_THRESHOLD = 1.0               # Treated if ratio < 1

# Dual Outcomes
OUTCOMES = {
    "ETS_CO2": ETS_CO2_COL,              # Verified Emissions (tCO2e)
    "Satellite_NOx": NOX_OUTCOME_COL,    # Beirle-style NOx (Yearly Mean kg/s)
}

# =============================================================================
# CONTROLS
# =============================================================================
# Base controls: applied to BOTH outcomes
CONTROLS = ["capacity_mw", "share_coal", "share_gas"]

# NOTE: AlphaEarth embeddings (emb_*) are stored in beirle_panel.parquet and are
# automatically added as controls for NOx outcome ONLY by run_dual_outcome_analysis().
# They are NOT used for ETS CO₂ because:
#   1. ETS CO₂ is administrative data - geographic context doesn't affect measurement
#   2. Geographic confounders are absorbed by Facility FE and Region×Year FE
#   3. Embeddings would control for something irrelevant to the ETS measurement
# See DAG: G_i → Y^NOx only, not G_i → Y^CO2

# Clustering: NUTS2 regions (primary)
# PyPSA clusters available for electricity heterogeneity via PYPSA_CLUSTER_COL

# Sample restrictions
MIN_YEARS_PER_FACILITY = 3

# Satellite-specific filters
SATELLITE_DETECTION_LIMIT = "conservative"  # "conservative" (0.11 kg/s) or "permissive" (0.03 kg/s)

# Heterogeneity dimensions
HETEROGENEITY_VARS = {
    "urbanization": IN_URBAN_AREA_COL,    # Urban vs rural
    "electricity": IS_ELECTRICITY_COL,     # Electricity generators (activity codes 1, 20)
    "fuel_coal": "share_coal",             # High coal (>0.5) vs other
    "capacity": "capacity_mw",             # Above/below median
}

print("Parameters configured:")
print(f"  Years: {START_YEAR}-{END_YEAR}")
print(f"  Treatment: {TREATMENT_COL} < {TREATMENT_THRESHOLD}")
print(f"  Outcomes: {list(OUTCOMES.keys())}")
print(f"  Base controls: {CONTROLS}")
print(f"  NOx-only controls: AlphaEarth embeddings (64 dims, auto-added)")
print(f"  Primary clustering: NUTS2 regions ({CLUSTER_COL})")
print(f"  Satellite DL: {SATELLITE_DETECTION_LIMIT}")
print(f"  Data paths: {DATA_DIR}")

In [ ]:
# =============================================================================
# Load Data (ETS + Satellite outcomes merged)
# =============================================================================
from data import load_analysis_panel, load_facilities_static

# Load static attributes (includes is_electricity, nuts2_region, etc.)
static = load_facilities_static(STATIC_PATH)

# Load panel with satellite outcomes included
panel = load_analysis_panel(
    static_path=STATIC_PATH,
    yearly_path=YEARLY_PATH,
    beirle_path=BEIRLE_PATH,
    include_satellite=True
)

print(f"\nPanel: {len(panel)} obs, {panel[FAC_ID_COL].nunique()} facilities")
print(f"Years: {panel[YEAR_COL].min()} - {panel[YEAR_COL].max()}")

print(f"\nStatic attributes:")
print(f"  is_electricity: {static[IS_ELECTRICITY_COL].sum()} / {len(static)} are electricity generators")
if CLUSTER_COL in static.columns:
    print(f"  {CLUSTER_COL}: {static[CLUSTER_COL].nunique()} unique regions")

print(f"\nAvailable outcomes:")
print(f"  ETS CO₂: {ETS_CO2_COL} → {panel[ETS_CO2_COL].notna().sum()} valid obs")
if NOX_OUTCOME_COL in panel.columns:
    print(f"  Satellite NOx: {NOX_OUTCOME_COL} → {panel[NOX_OUTCOME_COL].notna().sum()} valid obs")

## 2. Data Preparation and Treatment Construction

In [ ]:
# =============================================================================
# Build Treatment Variables
# =============================================================================
from data import build_treatment_variables, apply_sample_filters

# Continuous: eu_alloc_ratio (already in data)
# Discrete: treated = 1 if ratio < 1, cohort = first year treated
panel = build_treatment_variables(
    panel, 
    treatment_col=TREATMENT_COL,
    threshold=TREATMENT_THRESHOLD
)

# Apply sample filters for ETS CO₂ analysis
# NOTE: apply_ets_filter=True applies the emissions filter (≥100 ktCO₂/yr)
# This filter is specific to ETS CO₂ outcome and should NOT be used for satellite NOx
panel = apply_sample_filters(
    panel, 
    min_years=MIN_YEARS_PER_FACILITY,
    year_range=(START_YEAR, END_YEAR),
    require_outcome=True,
    outcome_col=ETS_CO2_COL,
    apply_ets_filter=True,  # Apply emissions filter for ETS CO₂ analysis
)

# Map is_electricity from static to panel
if IS_ELECTRICITY_COL in static.columns and IS_ELECTRICITY_COL not in panel.columns:
    elec_map = static.set_index(FAC_ID_COL)[IS_ELECTRICITY_COL]
    panel[IS_ELECTRICITY_COL] = panel[FAC_ID_COL].map(elec_map)
    print(f"\nMapped is_electricity to panel: {panel[IS_ELECTRICITY_COL].sum()} electricity facility-years")

In [ ]:
# =============================================================================
# Panel Summary
# =============================================================================
print(f"\nFinal panel:")
print(f"  Observations: {len(panel)}")
print(f"  Facilities: {panel[FAC_ID_COL].nunique()}")
print(f"  Years: {panel[YEAR_COL].min()}-{panel[YEAR_COL].max()}")
print(f"  Treated (ever): {(panel.groupby(FAC_ID_COL)['treated'].max() > 0).sum()}")
print(f"  Never treated: {(panel.groupby(FAC_ID_COL)['treated'].max() == 0).sum()}")

## 3. Assign NUTS2 Regions for Clustering

NUTS2 regions are Eurostat's standard geographic units for regional analysis. We use them for:
- **Clustered standard errors** (account for within-region correlation)  
- **Region × Year fixed effects** (absorb regional time-varying confounders)

The `nuts2_region` column should already be in `facilities_static` from the Data Pipeline. If not present, we fall back to `country_code`.

In [ ]:
# =============================================================================
# Assign NUTS2 Regions (from static data or fallback to country_code)
# =============================================================================
from clusters import assign_nuts_regions

# Check if nuts2_region is already in static data (from Data Pipeline)
if CLUSTER_COL in static.columns:
    print(f"NUTS2 region found in static data: {static[CLUSTER_COL].nunique()} unique regions")
    # Map to panel
    nuts_map = static.set_index(FAC_ID_COL)[CLUSTER_COL]
    panel[CLUSTER_COL] = panel[FAC_ID_COL].map(nuts_map)
else:
    print(f"NUTS2 region not found in static data, attempting spatial assignment...")
    try:
        panel = assign_nuts_regions(panel, static, cluster_col=CLUSTER_COL)
    except FileNotFoundError as e:
        print(f"NUTS regions file not found: {e}")
        print("Falling back to country_code as cluster variable")
        country_map = static.set_index(FAC_ID_COL)["country_code"]
        panel[CLUSTER_COL] = panel[FAC_ID_COL].map(country_map)

# Report cluster distribution
print(f"\nCluster summary ({CLUSTER_COL}):")
print(f"  Total observations: {len(panel)}")
print(f"  Unique clusters: {panel[CLUSTER_COL].nunique()}")
print(f"  Facilities per cluster (mean): {len(panel) / panel[CLUSTER_COL].nunique():.1f}")

# Show top clusters
print(f"\nTop 10 clusters by observation count:")
display(panel[CLUSTER_COL].value_counts().head(10))

## 4. TWFE Continuous Specifications

Two TWFE specifications for each outcome:
1. **Spec 1**: Facility + Year FE, clustered SEs by PyPSA region
2. **Spec 2**: Facility + Region×Year FE, clustered SEs by PyPSA region

In [ ]:
# =============================================================================
# Run Dual-Outcome TWFE Analysis
# =============================================================================
from continuous import run_dual_outcome_analysis, format_dual_results_table
from data import get_intersection_sample

# Run TWFE on FULL ETS sample (ground truth)
print("\n" + "=" * 70)
print("FULL ETS SAMPLE (ground truth CO₂)")
print("=" * 70)
dual_results_full = run_dual_outcome_analysis(
    panel,
    treatment_col=TREATMENT_COL,
    controls=CONTROLS,
    cluster_col=CLUSTER_COL,
    ets_col=LOG_ETS_CO2_COL,
    nox_col=NOX_OUTCOME_COL
)

# Display results table
print("\n" + "=" * 60)
print("TWFE Results Summary (Full Sample)")
print("=" * 60)
display(format_dual_results_table(dual_results_full))

## 4b. Embedding Dimensionality Reduction Comparison

For the satellite NOx outcome, we control for AlphaEarth embeddings (64 dimensions) to account for
systematic measurement heterogeneity. Two dimensionality reduction strategies are compared:

1. **PCA (unsupervised)**: Maximizes variance in embedding space. Safe for causal inference.
2. **Facility-level PLS (supervised)**: Projects onto directions predictive of mean NOx.

**Critical design choice**: PLS is trained on **facility-level means** (cross-sectional), not panel
observations. This prevents outcome snooping / regularization bias that would arise from training
on the same panel used for TWFE estimation. See Chernozhukov et al. (2018) on regularization bias
and Cinelli et al. (2022) on bad controls.

The facility-level projection is time-invariant and thus orthogonal to within-facility treatment
variation, making the reduced embeddings equivalent to pre-treatment covariates.

In [ ]:
# =============================================================================
# Embedding Dimensionality Reduction Comparison
# =============================================================================
from embedding_reduction import compare_reduction_methods, reduce_embeddings
from continuous import run_dual_outcome_analysis, format_dual_results_table

# Compare PCA vs PLS across different component counts
print("=" * 70)
print("EMBEDDING REDUCTION COMPARISON")
print("=" * 70)

comparison_df = compare_reduction_methods(
    panel,
    target_col=NOX_OUTCOME_COL,
    n_components_list=[5, 10, 15],
    emb_prefix="emb_"
)
display(comparison_df.pivot(index="n_components", columns="method", values="value"))

In [ ]:
# =============================================================================
# TWFE with Different Embedding Reduction Strategies
# =============================================================================

# Baseline: Raw embeddings (64 dims)
print("\n" + "=" * 70)
print("SPECIFICATION 1: Raw Embeddings (64 dims)")
print("=" * 70)
results_raw = run_dual_outcome_analysis(
    panel,
    treatment_col=TREATMENT_COL,
    controls=CONTROLS,
    cluster_col=CLUSTER_COL,
    ets_col=LOG_ETS_CO2_COL,
    nox_col=NOX_OUTCOME_COL,
    embedding_reduction=None  # Raw 64-dim embeddings
)

# PCA reduction (10 components)
print("\n" + "=" * 70)
print("SPECIFICATION 2: PCA-Reduced Embeddings (10 dims)")
print("=" * 70)
results_pca = run_dual_outcome_analysis(
    panel.copy(),  # Fresh copy to avoid column conflicts
    treatment_col=TREATMENT_COL,
    controls=CONTROLS,
    cluster_col=CLUSTER_COL,
    ets_col=LOG_ETS_CO2_COL,
    nox_col=NOX_OUTCOME_COL,
    embedding_reduction="pca",
    embedding_n_components=10
)

# PLS reduction (10 components, trained on facility-level mean NOx)
print("\n" + "=" * 70)
print("SPECIFICATION 3: Facility-Level PLS Embeddings (10 dims)")
print("=" * 70)
results_pls = run_dual_outcome_analysis(
    panel.copy(),  # Fresh copy to avoid column conflicts
    treatment_col=TREATMENT_COL,
    controls=CONTROLS,
    cluster_col=CLUSTER_COL,
    ets_col=LOG_ETS_CO2_COL,
    nox_col=NOX_OUTCOME_COL,
    embedding_reduction="pls",
    embedding_n_components=10
)

In [ ]:
# =============================================================================
# Compare NOx Coefficients Across Embedding Strategies
# =============================================================================
print("\n" + "=" * 70)
print("EMBEDDING REDUCTION ROBUSTNESS CHECK: Satellite NOx Coefficients")
print("=" * 70)

emb_comparison = []
for label, results in [("Raw (64 dims)", results_raw), 
                        ("PCA (10 dims)", results_pca), 
                        ("PLS (10 dims)", results_pls)]:
    if "satellite" in results and "coefficient" in results["satellite"]:
        sat = results["satellite"]
        emb_comparison.append({
            "Embedding Strategy": label,
            "Coefficient": sat["coefficient"],
            "SE": sat["se"],
            "P-value": sat["pvalue"],
            "N": sat["n_obs"]
        })

emb_comparison_df = pd.DataFrame(emb_comparison)
display(emb_comparison_df)

# Interpretation
print("\n### Interpretation")
print("If coefficients are stable across embedding strategies, results are robust")
print("to the choice of dimensionality reduction method.")
print("\nNote: PLS uses facility-level mean NOx for training (not panel observations)")
print("to prevent outcome snooping / regularization bias (Chernozhukov et al., 2018).")

In [ ]:
# =============================================================================
# Run TWFE on INTERSECTION Sample (both outcomes valid, above detection limit)
# =============================================================================

# Get intersection sample with satellite filters
# satellite_sample="significant" applies above_dl_0_11 AND rel_err_stat_lt_0_3 flags
intersection_panel = get_intersection_sample(
    panel,
    ets_col=ETS_CO2_COL,
    nox_col=NOX_OUTCOME_COL,
    satellite_sample="significant" if SATELLITE_DETECTION_LIMIT == "conservative" else "all"
)

if len(intersection_panel) > 50:
    print("\n" + "=" * 70)
    print("INTERSECTION SAMPLE (both ETS CO2 and Satellite NOx valid)")
    print("=" * 70)
    
    dual_results_intersection = run_dual_outcome_analysis(
        intersection_panel,
        treatment_col=TREATMENT_COL,
        controls=CONTROLS,
        cluster_col=CLUSTER_COL,
        ets_col=LOG_ETS_CO2_COL,
        nox_col=NOX_OUTCOME_COL
    )
    
    print("\n" + "=" * 60)
    print("TWFE Results Summary (Intersection Sample)")
    print("=" * 60)
    display(format_dual_results_table(dual_results_intersection))
else:
    print(f"Intersection sample too small ({len(intersection_panel)} obs)")

## 5. Discrete Treatment: Callaway-Sant'Anna DiD

Estimate treatment effects using the Callaway & Sant'Anna (2021) estimator for 
staggered difference-in-differences with heterogeneous treatment timing.

**Treatment event**: First year when `eu_alloc_ratio < 1`

Reference: [Callaway & Sant'Anna (2021)](https://psantanna.com/files/Callaway_SantAnna_2020.pdf)

In [ ]:
# =============================================================================
# Validate Cohorts for Callaway-Sant'Anna
# =============================================================================
from csdid import validate_cohorts

# Ensure cohorts are properly constructed (already done in build_treatment_variables)
cohort_validation = validate_cohorts(panel, cohort_col="cohort")

In [ ]:
# =============================================================================
# Callaway-Sant'Anna Estimation (All 6 Specifications)
# =============================================================================
from csdid import run_csdid_both_specs
from embedding_reduction import reduce_embeddings

csdid_results = {}

# --- Specification 1-2: ETS CO₂ (no embeddings) ---
print(f"\n{'#' * 60}")
print(f"# CS-DiD: ETS CO₂ (Specs 1-2)")
print(f"{'#' * 60}")

try:
    csdid_results["ETS_CO2"] = run_csdid_both_specs(
        panel,
        outcome_col=ETS_CO2_COL,
        cohort_col="cohort",
        cluster_col=CLUSTER_COL,
        control_group="nevertreated",
        est_method="dr"
    )
except Exception as e:
    print(f"Error running CS-DiD for ETS CO₂: {e}")
    csdid_results["ETS_CO2"] = None

# --- Specification 3-4: Satellite NOx with PCA embeddings ---
print(f"\n{'#' * 60}")
print(f"# CS-DiD: Satellite NOx + PCA (Specs 3-4)")
print(f"{'#' * 60}")

try:
    # Apply PCA reduction
    panel_pca = reduce_embeddings(
        panel.copy(),
        method="pca",
        n_components=10,
        emb_prefix="emb_"
    )
    csdid_results["Satellite_NOx_PCA"] = run_csdid_both_specs(
        panel_pca,
        outcome_col=NOX_OUTCOME_COL,
        cohort_col="cohort",
        cluster_col=CLUSTER_COL,
        control_group="nevertreated",
        est_method="dr"
    )
except Exception as e:
    print(f"Error running CS-DiD for NOx+PCA: {e}")
    csdid_results["Satellite_NOx_PCA"] = None

# --- Specification 5-6: Satellite NOx with PLS embeddings ---
print(f"\n{'#' * 60}")
print(f"# CS-DiD: Satellite NOx + PLS (Specs 5-6)")
print(f"{'#' * 60}")

try:
    # Apply PLS reduction (trained on facility-level mean NOx)
    panel_pls = reduce_embeddings(
        panel.copy(),
        method="pls",
        n_components=10,
        target_col=NOX_OUTCOME_COL,
        emb_prefix="emb_"
    )
    csdid_results["Satellite_NOx_PLS"] = run_csdid_both_specs(
        panel_pls,
        outcome_col=NOX_OUTCOME_COL,
        cohort_col="cohort",
        cluster_col=CLUSTER_COL,
        control_group="nevertreated",
        est_method="dr"
    )
except Exception as e:
    print(f"Error running CS-DiD for NOx+PLS: {e}")
    csdid_results["Satellite_NOx_PLS"] = None

In [ ]:
# =============================================================================
# Event Study Plots
# =============================================================================
from csdid import plot_event_study, test_pre_trends

for outcome_name, results in csdid_results.items():
    if results is None:
        continue
    
    # Use the no_cluster result for event study
    res = results.get("no_cluster")
    if res is None:
        continue
    
    print(f"\n{outcome_name} Event Study:")
    fig = plot_event_study(
        res,
        title=f"Event Study: ETS → {outcome_name}",
        max_pre=3,
        max_post=4
    )
    plt.show()
    
    # Pre-trends test
    test_pre_trends(res, n_pre_periods=2)

## 6. Covariate Balance and Diagnostics

In [ ]:
# =============================================================================
# Covariate Summary by Treatment Status
# =============================================================================
# Simple balance check
covariates = ["capacity_mw", "share_coal", "share_gas", "share_oil", "share_biomass"]
available_covs = [c for c in covariates if c in panel.columns]

print("Mean by treatment status (baseline year):")
baseline = panel.groupby(FAC_ID_COL).first()
print(baseline.groupby("treated")[available_covs].mean().T)

In [ ]:
# =============================================================================
# Outcome Trends: Treated vs Never Treated
# =============================================================================
for outcome_name, outcome_col in OUTCOMES.items():
    if outcome_col not in panel.columns:
        continue
    
    # Mean outcome by year and treatment status
    trends = panel.groupby([YEAR_COL, "treated"])[outcome_col].mean().unstack()
    
    fig, ax = plt.subplots(figsize=(10, 5))
    trends.plot(ax=ax, marker='o')
    ax.set_xlabel("Year")
    ax.set_ylabel(outcome_name)
    ax.set_title(f"{outcome_name} Trends: Treated vs Never-Treated")
    ax.legend(["Never Treated", "Ever Treated"])
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 6b. Outcome Correlation Diagnostic

Check correlation between ETS CO₂ (ground truth) and Satellite NOx (proxy).
- Positive correlation suggests both capture combustion activity
- Correlation strength indicates signal-to-noise in satellite proxy

In [ ]:
# =============================================================================
# ETS CO₂ vs Satellite NOx Correlation
# =============================================================================
from scipy import stats

# Get intersection sample (both outcomes valid)
corr_df = panel.dropna(subset=[LOG_ETS_CO2_COL, NOX_OUTCOME_COL])

if len(corr_df) > 10:
    # Compute correlations
    pearson_r, pearson_p = stats.pearsonr(corr_df[LOG_ETS_CO2_COL], corr_df[NOX_OUTCOME_COL])
    spearman_r, spearman_p = stats.spearmanr(corr_df[LOG_ETS_CO2_COL], corr_df[NOX_OUTCOME_COL])
    
    print(f"Outcome Correlation (N = {len(corr_df)} obs):")
    print(f"  Pearson r  = {pearson_r:.3f} (p = {pearson_p:.4f})")
    print(f"  Spearman ρ = {spearman_r:.3f} (p = {spearman_p:.4f})")
    
    # Scatter plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.scatter(corr_df[LOG_ETS_CO2_COL], corr_df[NOX_OUTCOME_COL], 
               alpha=0.4, s=20, c='steelblue')
    
    # Fit line
    slope, intercept, _, _, _ = stats.linregress(corr_df[LOG_ETS_CO2_COL], corr_df[NOX_OUTCOME_COL])
    x_line = np.linspace(corr_df[LOG_ETS_CO2_COL].min(), corr_df[LOG_ETS_CO2_COL].max(), 100)
    ax.plot(x_line, slope * x_line + intercept, 'r-', lw=2, label=f'OLS: β={slope:.4f}')
    
    ax.set_xlabel('Log ETS CO₂ (tCO₂/yr)')
    ax.set_ylabel('Satellite NOx (kg/s)')
    ax.set_title(f'ETS CO₂ vs Satellite NOx Correlation\\n(Pearson r = {pearson_r:.3f})')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # By fuel type (if available)
    if 'share_coal' in panel.columns:
        coal_mask = corr_df['share_coal'] > 0.5
        if coal_mask.sum() > 10 and (~coal_mask).sum() > 10:
            r_coal, _ = stats.pearsonr(corr_df.loc[coal_mask, LOG_ETS_CO2_COL], 
                                        corr_df.loc[coal_mask, NOX_OUTCOME_COL])
            r_other, _ = stats.pearsonr(corr_df.loc[~coal_mask, LOG_ETS_CO2_COL], 
                                         corr_df.loc[~coal_mask, NOX_OUTCOME_COL])
            print(f"\\nCorrelation by fuel type:")
            print(f"  Coal-dominated (>50%): r = {r_coal:.3f} (N = {coal_mask.sum()})")
            print(f"  Other fuels:           r = {r_other:.3f} (N = {(~coal_mask).sum()})")
else:
    print("Insufficient data for correlation analysis")

## 7. Heterogeneity Analysis

Examine whether treatment effects vary systematically across facility characteristics:

### Dimensions
1. **Electricity Sector** (`is_electricity`): EU ETS activity codes 1 or 20 (combustion)
   - Electricity generators face different market dynamics (dispatch optimization)
   - PyPSA-Eur clusters can be used for within-electricity-sector heterogeneity
2. **Urbanization**: Urban (SMOD ≥21) vs Rural facilities
   - Urban facilities have higher background NO₂, may attenuate satellite signal
3. **Fuel Type**: Coal-dominated (>50%) vs Other
   - Coal plants have higher NOx/CO₂ ratios and different abatement options
4. **Capacity**: Above vs below median capacity
   - Large facilities may have economies of scale in abatement
5. **Country**: Top emitting countries
   - Cross-country variation in policy enforcement

### Interpretation Notes
- Heterogeneous effects suggest treatment effect varies across groups
- Confidence intervals overlapping zero don't imply no effect (low power)
- Main interest: **direction** and **magnitude** consistency across subgroups

In [ ]:
# =============================================================================
# Heterogeneity Analysis: Split-Sample TWFE
# =============================================================================
from continuous import run_outcome_models

def run_heterogeneity_analysis(
    panel_df: pd.DataFrame,
    split_var: str,
    split_label: str,
    outcome_col: str = LOG_ETS_CO2_COL,
    treatment_col: str = TREATMENT_COL,
    controls: list = CONTROLS,
    cluster_col: str = CLUSTER_COL
):
    """Run TWFE on subsamples defined by split_var."""
    results = {}
    
    # Get unique groups
    if panel_df[split_var].dtype == bool or panel_df[split_var].nunique() == 2:
        groups = [True, False]
        labels = [f"{split_label}=True", f"{split_label}=False"]
    else:
        # For continuous, split at median
        median = panel_df[split_var].median()
        panel_df = panel_df.copy()
        panel_df["_split_group"] = panel_df[split_var] >= median
        split_var = "_split_group"
        groups = [True, False]
        labels = [f"{split_label}>=median", f"{split_label}<median"]
    
    for group, label in zip(groups, labels):
        subset = panel_df[panel_df[split_var] == group]
        if len(subset) < 50 or subset[FAC_ID_COL].nunique() < 10:
            print(f"  {label}: Insufficient data ({len(subset)} obs, {subset[FAC_ID_COL].nunique()} facilities)")
            continue
        
        try:
            res = run_outcome_models(
                subset,
                outcome_col=outcome_col,
                treatment_col=treatment_col,
                controls=controls,
                cluster_col=cluster_col
            )
            results[label] = res
            print(f"  {label}: N={len(subset)}, n_fac={subset[FAC_ID_COL].nunique()}")
        except Exception as e:
            print(f"  {label}: Error - {e}")
    
    return results

# =============================================================================
# Run heterogeneity analysis for ETS CO2 outcome
# =============================================================================
print("=" * 60)
print("HETEROGENEITY ANALYSIS: ETS CO2")
print("=" * 60)

het_results = {}

# 0. Electricity sector (from EU ETS activity codes 1, 20)
if IS_ELECTRICITY_COL in panel.columns:
    print(f"\n### Electricity Sector ({IS_ELECTRICITY_COL})")
    n_elec = panel[panel[IS_ELECTRICITY_COL] == True][FAC_ID_COL].nunique()
    n_other = panel[panel[IS_ELECTRICITY_COL] == False][FAC_ID_COL].nunique()
    print(f"  Electricity generators: {n_elec} facilities")
    print(f"  Other sectors: {n_other} facilities")
    het_results["electricity"] = run_heterogeneity_analysis(
        panel, IS_ELECTRICITY_COL, "Electricity"
    )

# 1. Urbanization
if IN_URBAN_AREA_COL in panel.columns:
    print(f"\n### Urbanization ({IN_URBAN_AREA_COL})")
    het_results["urbanization"] = run_heterogeneity_analysis(
        panel, IN_URBAN_AREA_COL, "Urban"
    )

# 2. Fuel type (coal-dominated)
if "share_coal" in panel.columns:
    print("\n### Fuel Type (Coal > 50%)")
    panel["_coal_dominated"] = panel["share_coal"] > 0.5
    het_results["fuel_coal"] = run_heterogeneity_analysis(
        panel, "_coal_dominated", "Coal>50%"
    )

# 3. Capacity (above/below median)
if "capacity_mw" in panel.columns:
    print("\n### Capacity (Above/Below Median)")
    het_results["capacity"] = run_heterogeneity_analysis(
        panel, "capacity_mw", "Capacity"
    )

# 4. Top countries
if "country_code" in panel.columns:
    print("\n### Country (Top 3 by facilities)")
    top_countries = panel.groupby("country_code")[FAC_ID_COL].nunique().nlargest(3).index
    for country in top_countries:
        subset = panel[panel["country_code"] == country]
        if len(subset) >= 50:
            try:
                res = run_outcome_models(
                    subset,
                    outcome_col=LOG_ETS_CO2_COL,
                    treatment_col=TREATMENT_COL,
                    controls=CONTROLS,
                    cluster_col=CLUSTER_COL
                )
                het_results[f"country_{country}"] = res
                print(f"  {country}: N={len(subset)}, n_fac={subset[FAC_ID_COL].nunique()}")
            except Exception as e:
                print(f"  {country}: Error - {e}")

## 7. Robustness Checks

### Methodological Notes

**Clustering SEs vs Region FE:**
- **Clustered SEs** (by PyPSA region or k-means cluster): Accounts for within-cluster correlation in errors. Appropriate when units within clusters are correlated but treatment varies within clusters.
- **Region FE** (country × year): Absorbs region-specific time-varying shocks. Use when regional factors (electricity prices, policy enforcement) affect both treatment and outcome.

**Both approaches are valid but address different concerns:**
- Clustered SEs: Inference problem (correct standard errors)
- Region FE: Identification problem (absorb confounders)

We run both separately.

In [ ]:
# =============================================================================
# Summary: Dual-Outcome Results
# =============================================================================
print("\\n" + "=" * 70)
print("FINAL RESULTS SUMMARY: DUAL OUTCOMES")
print("=" * 70)

# TWFE Results
print("\\n### TWFE Continuous Treatment ###")
if 'dual_results_full' in dir():
    display(format_dual_results_table(dual_results_full))

# CS-DiD Results (if run)
print("\\n### Callaway-Sant'Anna Discrete Treatment ###")
if 'csdid_results' in dir() and csdid_results:
    for outcome_name, results in csdid_results.items():
        if results is None:
            print(f"{outcome_name}: Not estimated (csdid package required)")
            continue
        
        for spec_name, res in results.items():
            att = res["agg_simple"]["att"]
            se = res["agg_simple"]["se"]
            print(f"{outcome_name} ({spec_name}): ATT = {att:.6f} (SE = {se:.6f})")

## 8. Summary and Next Steps

### Specifications Run

**TWFE Continuous (per outcome):**
1. Facility + Year FE, region-clustered SEs
2. Facility + Region×Year FE, region-clustered SEs

**Callaway-Sant'Anna Discrete (per outcome):**
1. Unit-level SEs
2. Region-clustered SEs

**Dual Outcomes:**
- **ETS CO₂**: `log_ets_co2` — log verified emissions (ground truth)
- **Satellite NOx**: `beirle_nox_kg_s` — Beirle-style flux-divergence proxy

### Interpretation Notes

1. **ETS CO₂ effects** are more reliable (ground-truth administrative data)
2. **Satellite NOx effects** may be attenuated due to ~35-45% measurement error
3. **Consistent signs** across outcomes strengthen causal claims
4. **Detection limit filter** (0.11 kg/s conservative) may introduce selection

### Next Steps
1. **Run Data Pipeline.ipynb** to generate satellite outcome data
2. **Install packages**: `pip install pyfixest csdid geopandas`
3. **Heterogeneity analysis** by fuel type, country, capacity
4. **Sensitivity analysis** with different treatment thresholds
5. **Compare ETS vs NOx** on intersection sample